In [9]:
color_mapping = {
    0: "#F0F8FF", 1: "#FAEBD7", 2: "#00FFFF", 3: "#7FFFD4", 4: "#F0FFFF", 5: "#F5F5DC", 6: "#FFE4C4", 
    7: "#000000", 8: "#FFEBCD", 9: "#0000FF", 10: "#8A2BE2", 11: "#A52A2A", 12: "#DEB887", 13: "#5F9EA0", 
    14: "#7FFF00", 15: "#D2691E", 16: "#FF7F50", 17: "#6495ED", 18: "#FFF8DC", 19: "#DC143C", 20: "#00FFFF", 
    21: "#00008B", 22: "#008B8B", 23: "#B8860B", 24: "#A9A9A9", 25: "#006400", 26: "#BDB76B", 27: "#8B008B", 
    28: "#556B2F", 29: "#FF8C00", 30: "#9932CC", 31: "#8B0000", 32: "#E9967A", 33: "#8FBC8F", 34: "#483D8B", 
    35: "#2F4F4F", 36: "#00CED1", 37: "#9400D3", 38: "#FF1493", 39: "#00BFFF", 40: "#696969", 41: "#1E90FF", 
    42: "#B22222", 43: "#FFFAF0", 44: "#228B22", 45: "#FF00FF", 46: "#DCDCDC", 47: "#F8F8FF", 48: "#FFD700", 
    49: "#DAA520", 50: "#808080", 51: "#008000", 52: "#ADFF2F", 53: "#F0FFF0", 54: "#FF69B4", 55: "#CD5C5C", 
    56: "#4B0082", 57: "#FFFFF0", 58: "#F0E68C", 59: "#E6E6FA", 60: "#FFF0F5", 61: "#7CFC00", 62: "#FFFACD", 
    63: "#ADD8E6", 64: "#F08080", 65: "#E0FFFF", 66: "#FAFAD2", 67: "#90EE90", 68: "#D3D3D3", 69: "#FFB6C1", 
    70: "#FFA07A", 71: "#20B2AA", 72: "#87CEFA", 73: "#778899", 74: "#B0C4DE", 75: "#FFFFE0", 76: "#00FF00", 
    77: "#32CD32", 78: "#FAF0E6", 79: "#FF00FF", 80: "#800000", 81: "#66CDAA", 82: "#0000CD", 83: "#BA55D3", 
    84: "#9370DB", 85: "#3CB371", 86: "#7B68EE", 87: "#00FA9A", 88: "#48D1CC", 89: "#C71585", 90: "#191970", 
    91: "#F5FFFA", 92: "#FFE4E1", 93: "#FFE4B5", 94: "#FFDEAD", 95: "#000080", 96: "#FDF5E6", 97: "#808000", 
    98: "#6B8E23", 99: "#FFA500", 100: "#FF4500", 101: "#DA70D6", 102: "#EEE8AA", 103: "#98FB98", 104: "#AFEEEE", 
    105: "#DB7093", 106: "#FFEFD5", 107: "#FFDAB9", 108: "#CD853F", 109: "#FFC0CB", 110: "#DDA0DD", 111: "#B0E0E6", 
    112: "#800080", 113: "#FF0000", 114: "#BC8F8F", 115: "#4169E1", 116: "#8B4513", 117: "#FA8072", 118: "#FAA460", 
    119: "#2E8B57", 120: "#FFF5EE", 121: "#A0522D", 122: "#C0C0C0", 123: "#87CEEB", 124: "#6A5ACD", 125: "#708090", 
    126: "#FFFAFA", 127: "#00FF7F", 128: "#4682B4", 129: "#D2B48C", 130: "#008080", 131: "#D8BFD8", 132: "#FF6347", 
    133: "#40E0D0", 134: "#EE82EE", 135: "#F5DEB3", 136: "#FFFFFF", 137: "#F5F5F5", 138: "#FFFF00", 139: "#9ACD32"
}

In [10]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt


frames_root_dir = './raw_volvo_demo_video_frames/'
age_root_dir = './output/ages/'
results_root_dir = './face_detection_results/'
if not os.path.exists(age_root_dir):
    os.makedirs(age_root_dir)
    
frames = [ele for ele in os.listdir(frames_root_dir) if ele.endswith('.jpg')]


for frame_no in range(len(frames)):

    if frame_no < 2670:
        continue
    if frame_no > 2695:
        continue
        
    print('Processing frame {}'.format(frame_no))

    frame_img_path = os.path.join(frames_root_dir, '{}.jpg'.format(frame_no))
    detected_json_path = os.path.join(results_root_dir, '{}.json'.format(frame_no))

    image = cv2.imread(frame_img_path, cv2.IMREAD_COLOR)
    response = json.load(open(detected_json_path, 'r'))
    
#     if len(response['FaceDetails']) == 0:
#         continue
    
    male_count = 0
    female_count = 0
    
    plt.figure(figsize=(9, 5))
    for face_obj in response['FaceDetails']:
        age_high = face_obj['AgeRange']['High']
        age_low = face_obj['AgeRange']['Low']
        
        gender = face_obj['Gender']['Value']
        gender_confidence = face_obj['Gender']['Confidence']        
        
        print('\tAge: {}~{}, gender = {}, prob = {}'.format(age_low, age_high, gender, gender_confidence))
    
        mu = 0.5 * (age_high + age_low)
        sigma = (age_high - age_low) / 4.0
        age_dist = mu + sigma * np.random.randn(2000)
        
        n, bins, patches = plt.hist(age_dist, int(np.max(age_dist) - np.min(age_dist)), 
                                    color=color_mapping[int(mu)], density=1.0, alpha=0.7)
        
    plt.axis([0, 100, 0, 0.3])    
    
    font = {
        'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size' : 18,
        }
    plt.tick_params(axis='both', which='major', labelsize=16)
    plt.xlabel('Age Range', font)
    plt.ylabel('Probability', font)
    plt.title('Histogram of Age Distribution', font)
    
    age_dist_dump_path = os.path.join(age_root_dir, '{}.jpg'.format(frame_no))
    plt.savefig(
        age_dist_dump_path,
        dpi=200, bbox_inches='tight', facecolor='#F0FFFF')
    plt.close()
    

Processing frame 2670
	Age: 23~35, gender = Female, prob = 99.95584106445312
	Age: 14~26, gender = Female, prob = 58.25126266479492
	Age: 31~47, gender = Male, prob = 97.83155059814453
	Age: 49~67, gender = Male, prob = 98.8743667602539
	Age: 48~66, gender = Male, prob = 98.18285369873047
	Age: 22~34, gender = Male, prob = 98.62841796875
	Age: 36~54, gender = Female, prob = 67.3689193725586
	Age: 24~38, gender = Female, prob = 61.76292419433594
Processing frame 2671
	Age: 23~35, gender = Female, prob = 99.95104217529297
	Age: 30~46, gender = Male, prob = 96.37841796875
	Age: 11~21, gender = Male, prob = 66.72901153564453
	Age: 49~67, gender = Male, prob = 99.07246398925781
	Age: 50~68, gender = Male, prob = 99.13465118408203
	Age: 22~34, gender = Male, prob = 99.37919616699219
	Age: 36~54, gender = Female, prob = 96.67129516601562
	Age: 26~42, gender = Female, prob = 75.87017059326172
Processing frame 2672
	Age: 26~42, gender = Female, prob = 99.95645904541016
	Age: 38~56, gender = Mal

In [6]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt

results_root_dir = './face_detection_results/'
frames_root_dir = './raw_volvo_demo_video_frames/'
gender_root_dir = './output/gender/'
if not os.path.exists(gender_root_dir):
    os.makedirs(gender_root_dir)
    
frames = [ele for ele in os.listdir(frames_root_dir) if ele.endswith('.jpg')]

male_cnt_queue, female_cnt_queue = list(), list()

for frame_no in range(len(frames)):
    print('Processing frame {}'.format(frame_no))

    frame_img_path = os.path.join(frames_root_dir, '{}.jpg'.format(frame_no))
    detected_json_path = os.path.join(results_root_dir, '{}.json'.format(frame_no))

    image = cv2.imread(frame_img_path, cv2.IMREAD_COLOR)
    response = json.load(open(detected_json_path, 'r'))
    
    window_size = 90
    
    male_count = 0
    female_count = 0
    
    for face_obj in response['FaceDetails']:
        gender = face_obj['Gender']['Value']
        gender_confidence = face_obj['Gender']['Confidence']
        
        if gender == 'Male':
            male_count += 1
        if gender == 'Female':
            female_count += 1
    
    if len(male_cnt_queue) < window_size:
        male_cnt_queue.append(male_count)
    else:
        del male_cnt_queue[0]
        male_cnt_queue.append(male_count)

    if len(female_cnt_queue) < window_size:
        female_cnt_queue.append(female_count)
    else:
        del female_cnt_queue[0]
        female_cnt_queue.append(female_count)
        
    plt.figure(figsize=(9, 5))
    font = {
        'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size' : 18,
        }
    
    if len(male_cnt_queue) < window_size:
        frames_seq = np.arange(0, len(male_cnt_queue))
    else:
        frames_seq = np.arange(frame_no - window_size, frame_no)
            
    l1, = plt.plot(frames_seq, male_cnt_queue, 'c^-', markersize=3)
    l2, = plt.plot(frames_seq, female_cnt_queue, 's-', color='#FF1493', markersize=3)
    
    plt.legend([
        'Male Counts',
        'Female Counts'
    ])
    
    plt.tick_params(axis='both', which='major', labelsize=16)
    plt.xlabel('Frame Index', font)
    plt.ylabel('Count of Male/Female', font)
    plt.title('Gender Distribution', font)
    
    if len(male_cnt_queue) < window_size:
        plt.axis([0, window_size, 0, 48])
        xticks = range(0, window_size, 1)
        
    else:
        plt.axis([frame_no - window_size, frame_no, 0, 48])
        xticks = range(frame_no - window_size, frame_no, 1)

    select_xticks = [ele for ele in xticks if ele % 10 == 0]
    plt.xticks(select_xticks[0:-1], select_xticks[0:-1]) 
    
    gender_dist_dump_path = os.path.join(gender_root_dir, '{}.jpg'.format(frame_no))
    plt.savefig(
        gender_dist_dump_path,
        dpi=200, bbox_inches='tight', facecolor='snow')
    
#     plt.show()
    plt.close()


Processing frame 0
Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39
Processing frame 40
Processing frame 41
Processing frame 42
Processing frame 43
Processing frame 44
Processing frame 45
Processing frame 46
Processing frame 47
Processing frame 48
Processing frame 49
Processing

Processing frame 396
Processing frame 397
Processing frame 398
Processing frame 399
Processing frame 400
Processing frame 401
Processing frame 402
Processing frame 403
Processing frame 404
Processing frame 405
Processing frame 406
Processing frame 407
Processing frame 408
Processing frame 409
Processing frame 410
Processing frame 411
Processing frame 412
Processing frame 413
Processing frame 414
Processing frame 415
Processing frame 416
Processing frame 417
Processing frame 418
Processing frame 419
Processing frame 420
Processing frame 421
Processing frame 422
Processing frame 423
Processing frame 424
Processing frame 425
Processing frame 426
Processing frame 427
Processing frame 428
Processing frame 429
Processing frame 430
Processing frame 431
Processing frame 432
Processing frame 433
Processing frame 434
Processing frame 435
Processing frame 436
Processing frame 437
Processing frame 438
Processing frame 439
Processing frame 440
Processing frame 441
Processing frame 442
Processing fr

Processing frame 787
Processing frame 788
Processing frame 789
Processing frame 790
Processing frame 791
Processing frame 792
Processing frame 793
Processing frame 794
Processing frame 795
Processing frame 796
Processing frame 797
Processing frame 798
Processing frame 799
Processing frame 800
Processing frame 801
Processing frame 802
Processing frame 803
Processing frame 804
Processing frame 805
Processing frame 806
Processing frame 807
Processing frame 808
Processing frame 809
Processing frame 810
Processing frame 811
Processing frame 812
Processing frame 813
Processing frame 814
Processing frame 815
Processing frame 816
Processing frame 817
Processing frame 818
Processing frame 819
Processing frame 820
Processing frame 821
Processing frame 822
Processing frame 823
Processing frame 824
Processing frame 825
Processing frame 826
Processing frame 827
Processing frame 828
Processing frame 829
Processing frame 830
Processing frame 831
Processing frame 832
Processing frame 833
Processing fr

Processing frame 1170
Processing frame 1171
Processing frame 1172
Processing frame 1173
Processing frame 1174
Processing frame 1175
Processing frame 1176
Processing frame 1177
Processing frame 1178
Processing frame 1179
Processing frame 1180
Processing frame 1181
Processing frame 1182
Processing frame 1183
Processing frame 1184
Processing frame 1185
Processing frame 1186
Processing frame 1187
Processing frame 1188
Processing frame 1189
Processing frame 1190
Processing frame 1191
Processing frame 1192
Processing frame 1193
Processing frame 1194
Processing frame 1195
Processing frame 1196
Processing frame 1197
Processing frame 1198
Processing frame 1199
Processing frame 1200
Processing frame 1201
Processing frame 1202
Processing frame 1203
Processing frame 1204
Processing frame 1205
Processing frame 1206
Processing frame 1207
Processing frame 1208
Processing frame 1209
Processing frame 1210
Processing frame 1211
Processing frame 1212
Processing frame 1213
Processing frame 1214
Processing

Processing frame 1543
Processing frame 1544
Processing frame 1545
Processing frame 1546
Processing frame 1547
Processing frame 1548
Processing frame 1549
Processing frame 1550
Processing frame 1551
Processing frame 1552
Processing frame 1553
Processing frame 1554
Processing frame 1555
Processing frame 1556
Processing frame 1557
Processing frame 1558
Processing frame 1559
Processing frame 1560
Processing frame 1561
Processing frame 1562
Processing frame 1563
Processing frame 1564
Processing frame 1565
Processing frame 1566
Processing frame 1567
Processing frame 1568
Processing frame 1569
Processing frame 1570
Processing frame 1571
Processing frame 1572
Processing frame 1573
Processing frame 1574
Processing frame 1575
Processing frame 1576
Processing frame 1577
Processing frame 1578
Processing frame 1579
Processing frame 1580
Processing frame 1581
Processing frame 1582
Processing frame 1583
Processing frame 1584
Processing frame 1585
Processing frame 1586
Processing frame 1587
Processing

...

Processing frame 3035
Processing frame 3036
Processing frame 3037
Processing frame 3038
Processing frame 3039
Processing frame 3040
Processing frame 3041
Processing frame 3042
Processing frame 3043
Processing frame 3044
Processing frame 3045
Processing frame 3046
Processing frame 3047
Processing frame 3048
Processing frame 3049
Processing frame 3050
Processing frame 3051
Processing frame 3052
Processing frame 3053
Processing frame 3054
Processing frame 3055
Processing frame 3056
Processing frame 3057
Processing frame 3058
Processing frame 3059
Processing frame 3060
Processing frame 3061
Processing frame 3062
Processing frame 3063
Processing frame 3064
Processing frame 3065
Processing frame 3066
Processing frame 3067
Processing frame 3068
Processing frame 3069
Processing frame 3070
Processing frame 3071
Processing frame 3072
Processing frame 3073
Processing frame 3074
Processing frame 3075
Processing frame 3076
Processing frame 3077
Processing frame 3078
Processing frame 3079
Processing

Processing frame 3408
Processing frame 3409
Processing frame 3410
Processing frame 3411
Processing frame 3412
Processing frame 3413
Processing frame 3414
Processing frame 3415
Processing frame 3416
Processing frame 3417
Processing frame 3418
Processing frame 3419
Processing frame 3420
Processing frame 3421
Processing frame 3422
Processing frame 3423
Processing frame 3424
Processing frame 3425
Processing frame 3426
Processing frame 3427
Processing frame 3428
Processing frame 3429
Processing frame 3430
Processing frame 3431
Processing frame 3432
Processing frame 3433
Processing frame 3434
Processing frame 3435
Processing frame 3436
Processing frame 3437
Processing frame 3438
Processing frame 3439
Processing frame 3440
Processing frame 3441
Processing frame 3442
Processing frame 3443
Processing frame 3444
Processing frame 3445
Processing frame 3446
Processing frame 3447
Processing frame 3448
Processing frame 3449
Processing frame 3450
Processing frame 3451
Processing frame 3452
Processing

Processing frame 3781
Processing frame 3782
Processing frame 3783
Processing frame 3784
Processing frame 3785
Processing frame 3786
Processing frame 3787
Processing frame 3788
Processing frame 3789
Processing frame 3790
Processing frame 3791
Processing frame 3792
Processing frame 3793
Processing frame 3794
Processing frame 3795
Processing frame 3796
Processing frame 3797
Processing frame 3798
Processing frame 3799
Processing frame 3800
Processing frame 3801
Processing frame 3802
Processing frame 3803
Processing frame 3804
Processing frame 3805
Processing frame 3806
Processing frame 3807
Processing frame 3808
Processing frame 3809
Processing frame 3810
Processing frame 3811
Processing frame 3812
Processing frame 3813
Processing frame 3814
Processing frame 3815
Processing frame 3816
Processing frame 3817
Processing frame 3818
Processing frame 3819
Processing frame 3820
Processing frame 3821
Processing frame 3822
Processing frame 3823
Processing frame 3824
Processing frame 3825
Processing

Processing frame 4154
Processing frame 4155
Processing frame 4156
Processing frame 4157
Processing frame 4158
Processing frame 4159
Processing frame 4160
Processing frame 4161
Processing frame 4162
Processing frame 4163
Processing frame 4164
Processing frame 4165
Processing frame 4166
Processing frame 4167
Processing frame 4168
Processing frame 4169
Processing frame 4170
Processing frame 4171
Processing frame 4172
Processing frame 4173
Processing frame 4174
Processing frame 4175
Processing frame 4176
Processing frame 4177
Processing frame 4178
Processing frame 4179
Processing frame 4180
Processing frame 4181
Processing frame 4182
Processing frame 4183
Processing frame 4184
Processing frame 4185
Processing frame 4186
Processing frame 4187
Processing frame 4188
Processing frame 4189
Processing frame 4190
Processing frame 4191
Processing frame 4192
Processing frame 4193
Processing frame 4194
Processing frame 4195
Processing frame 4196
Processing frame 4197
Processing frame 4198
Processing

Processing frame 4527
Processing frame 4528
Processing frame 4529
Processing frame 4530
Processing frame 4531
Processing frame 4532
Processing frame 4533
Processing frame 4534
Processing frame 4535
Processing frame 4536
Processing frame 4537
Processing frame 4538
Processing frame 4539
Processing frame 4540
Processing frame 4541
Processing frame 4542
Processing frame 4543
Processing frame 4544
Processing frame 4545
Processing frame 4546
Processing frame 4547
Processing frame 4548
Processing frame 4549
Processing frame 4550
Processing frame 4551
Processing frame 4552
Processing frame 4553
Processing frame 4554
Processing frame 4555
Processing frame 4556
Processing frame 4557
Processing frame 4558
Processing frame 4559
